# Scene Beat to Prose

Train the LLM to write paragraphs in my writing style, based on scene beats.

In [4]:
from mlx_lm import load, generate
from openai import OpenAI
import json
import os
from tqdm import tqdm
from prompts import world_explanation
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

def separate_chapters(text):
    chapters = []
    lines = text.split('\n')
    current_chapter = []
    weekdays = ['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag', 'Sonntag']
    
    for i, line in enumerate(lines):
        if i < len(lines) - 1 and any(lines[i+1].startswith(day) for day in weekdays):
            if current_chapter:
                chapters.append('\n'.join(current_chapter))
                current_chapter = []
        current_chapter.append(line)
    
    if current_chapter:
        chapters.append('\n'.join(current_chapter))
    
    return chapters

def chunk_chapters(chapters, max_words=500):
    chunked_chapters = []
    for chapter in chapters:
        lines = chapter.split('\n')
        current_chunk = []
        word_count = 0
        
        for line in lines:
            line_words = len(line.split())
            if word_count + line_words > max_words:
                if current_chunk:
                    chunked_chapters.append('\n'.join(current_chunk))
                current_chunk = [line]
                word_count = line_words
            else:
                current_chunk.append(line)
                word_count += line_words
        
        if current_chunk:
            chunked_chapters.append('\n'.join(current_chunk))
    
    return chunked_chapters

def paragraph_to_scene_beats(chunk, model, tokenizer, use_openai=False, openai_model="gpt-4o-mini"):
    prompt = f"""
    Du bist ein professioneller Autor und arbeitest an einer Fantasy-Szene für den Roman "ANSTURM": {world_explanation}
    
    Vor dir liegt ein Textparagraph, und deine Aufgabe ist es, die zugrundeliegenden Scene Beats zu rekonstruieren. Jeder Beat sollte die Motivationen der Charaktere, die äusseren Umstände und das Ziel der Szene zusammenfassen, aber auf einfache und knappe Weise.

    Hier ist ein Beispiel für Scene Beats, wie du sie schreiben sollst:

    - Sokrate rennt verzweifelt durch die Dunkelheit, verfolgt von den Gedanken, dass er zu spät kommen könnte, um das Leben, das er kennt, zu retten. Er ist erschöpft, doch der Drang, die gesammelten Informationen weiterzugeben, treibt ihn an.
    - Der Wald um ihn herum wird immer dunkler, der Mond geht auf, und damit erklingt das Heulen der Werwölfe in der Ferne. Er spürt die Bedrohung näherkommen, kämpft aber mit aller Kraft weiter. Die Schreie aus der Ferne holen ihn in die Realität zurück.
    - Vor dem Portal ergreift er das Amulett. Er weiss, dass er das Portal damit öffnen kann, indem er es hinhält. Kurz danach wird er von einem Werwolf zu Boden gerissen. Schmerz durchfährt seinen Körper. Er erkennt, dass es kein Entkommen mehr gibt.
    - Die Werwölfe umringen ihn, und der Anführer beißt zu. Mit dem letzten Gedanken an sein altes Ich, bevor er vollständig verwandelt wird, akzeptiert Sokrate, dass sein altes Leben endet.

    Der Paragraph:

    "{chunk}"

    Antworte nur mit den Scene Beat(s), ohne weitere Einleitung oder Erklärung.
    """

    try:
        if use_openai:
            response = client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                model=openai_model,
                max_tokens=1000
            )
            scene_beats = response.choices[0].message.content
        else:
            messages = [{"role": "user", "content": prompt}]
            prompt = tokenizer.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True
            )
            scene_beats = generate(model, tokenizer, prompt=prompt, verbose=False, max_tokens=1000)
        return scene_beats
    except Exception as e:
        print(f"Error generating rephrased text: {e}")
        return f"EXCEPTION: {e}"

# Function to generate scene beats for chunks
def generate_scene_beats(input_file, output_file):
    # Load existing results if any
    existing_results = {}
    if os.path.exists(output_file):
        with open(output_file, 'r') as f:
            for line in f:
                data = json.loads(line)
                existing_results[data['id']] = data

    # Process chunks and generate scene beats
    with open(input_file, 'r') as in_file, open(output_file, 'a') as out_file:
        for line in tqdm(in_file):
            data = json.loads(line)
            chunk_id = data['id']
            
            # Skip if already processed
            if chunk_id in existing_results:
                continue
            
            scene_beats = paragraph_to_scene_beats(data['chunk'], model, tokenizer, use_openai=True)
            print(f'Generated scene beats for chunk {chunk_id}')
            
            result = {
                "id": chunk_id,
                "chunk": data['chunk'],
                "scene_beats": scene_beats
            }
            
            json.dump(result, out_file)
            out_file.write('\n')
            out_file.flush()  # Ensure data is written immediately

CHUNK_CHAPTERS = False
CHUNK_SIZE = 250
VERSION = "v2"

# Load model and tokenizer
model, tokenizer = load("models/frdm-Llama-3.1-8B-Write")

# Read and process the text
text = open('data/ansturm.txt', 'r').read()
if CHUNK_CHAPTERS:
    chapters = separate_chapters(text)
    prose_paragraphs = chunk_chapters(chapters, max_words=CHUNK_SIZE)
else:
    prose_paragraphs = separate_chapters(text)

# Save chunked chapters to a JSONL file
chunked_file = f'data/chapters_{VERSION}.jsonl'
with open(chunked_file, 'w') as f:
    for i, chunk in enumerate(prose_paragraphs):
        json.dump({"id": i, "chunk": chunk}, f)
        f.write('\n')

# Generate scene beats
output_file = f'data/chapters_with_beats_{VERSION}.jsonl'
generate_scene_beats(chunked_file, output_file)

print("Scene beats generation completed.")

1it [00:02,  2.53s/it]

Generated scene beats for chunk 0


2it [00:06,  3.11s/it]

Generated scene beats for chunk 1


3it [00:08,  2.95s/it]

...

Generated scene beats for chunk 24


26it [01:12,  2.89s/it]

Generated scene beats for chunk 25


27it [01:15,  2.80s/it]

Generated scene beats for chunk 26
Scene beats generation completed.


In [5]:
import json

def create_dataset_entry(scene_beats, paragraph):
    messages = [
        {"role": "system", "content": "Du bist der Fantasy-Autor Yvo K. Jedes Mal, wenn ich dir Scene Beats vorschreibe, schreibst du die komplette Szene auf der Grundlage der Idee. Schliesse die Szene nicht selbst ab, sondern halte dich genau an die Scene Beats."},
        {"role": "user", "content": scene_beats},
        {"role": "assistant", "content": paragraph}
    ]
    return json.dumps({"messages": messages})

def create_dataset(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            data = json.loads(line)
            dataset_entry = create_dataset_entry(data['scene_beats'], data['chunk'])
            outfile.write(dataset_entry + '\n')

# Create the dataset
input_file = f'data/chapters_with_beats_{VERSION}.jsonl'
output_file = f'data/scene_beat_to_prose_dataset_{VERSION}.jsonl'
create_dataset(input_file, output_file)

print("Dataset creation completed.")

Dataset creation completed.


In [6]:
import os
import random

def create_train_valid_test_split(input_file, output_folder, train_ratio=0.8, valid_ratio=0.1, test_ratio=0.1, seed=42):
    # Ensure ratios sum to 1
    assert abs(train_ratio + valid_ratio + test_ratio - 1.0) < 1e-5, "Ratios must sum to 1"

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Read all lines from the input file
    with open(input_file, 'r') as infile:
        lines = infile.readlines()

    # Shuffle the lines
    random.seed(seed)
    random.shuffle(lines)

    # Calculate split indices
    train_split = int(len(lines) * train_ratio)
    valid_split = train_split + int(len(lines) * valid_ratio)

    # Split the data
    train_data = lines[:train_split]
    valid_data = lines[train_split:valid_split]
    test_data = lines[valid_split:]

    # Write train data
    train_file = os.path.join(output_folder, 'train.jsonl')
    with open(train_file, 'w') as outfile:
        outfile.writelines(train_data)

    # Write validation data
    valid_file = os.path.join(output_folder, 'valid.jsonl')
    with open(valid_file, 'w') as outfile:
        outfile.writelines(valid_data)

    # Write test data
    test_file = os.path.join(output_folder, 'test.jsonl')
    with open(test_file, 'w') as outfile:
        outfile.writelines(test_data)

    print(f"Train-validation-test split created in {output_folder}")
    print(f"Train samples: {len(train_data)}")
    print(f"Validation samples: {len(valid_data)}")
    print(f"Test samples: {len(test_data)}")

# Usage
dataset_name = f"frdm-Llama-3.1-8B-Write-Beat-to-Prose-{VERSION}"

input_file = f'data/scene_beat_to_prose_dataset_{VERSION}.jsonl'
output_folder = f'data/{dataset_name}'
create_train_valid_test_split(input_file, output_folder)

Train-validation-test split created in data/frdm-Llama-3.1-8B-Write-Beat-to-Prose-v2
Train samples: 21
Validation samples: 2
Test samples: 4


In [7]:
# Model inference
from mlx_lm import load, generate
import json

def generate_scene(model, tokenizer, scene_beats, max_tokens=4096, temp=0.5):
    messages = [
        {"role": "system", "content": "Du bist der Fantasy-Autor Yvo K. Jedes Mal, wenn ich dir einen Scene Beat vorschreibe, schreibst du die komplette Szene auf der Grundlage der Idee. Schliesse die Szene nicht selbst ab, sondern halte dich genau an die Scene Beats. Schliesse nicht mit einer Vorahnung ab."},
        {"role": "user", "content": scene_beats}
    ]

    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    return generate(model, tokenizer, prompt=json.dumps(prompt), max_tokens=max_tokens, temp=temp, verbose=True)

# Load models
raw_model, raw_tokenizer = load("models/frdm-Llama-3.1-8B-Write")
fine_tuned_model, fine_tuned_tokenizer = load("models/frdm-Llama-3.1-8B-Write", adapter_path="models/frdm-Llama-3.1-8B-Write-Beat-to-Prose-v2")

# Scene beats
scene_beats = """
- Raven kniet verzweifelt im Schlamm und versucht, Caleor aus seiner Ohnmacht zu holen. Die Angst um Cale und die bevorstehende Gefahr treibt ihn an, w\u00e4hrend er gleichzeitig von der dr\u00e4ngenden Realit\u00e4t \u00fcberw\u00e4ltigt ist.\n- Caleor erwacht, panisch und desorientiert, konfrontiert mit der erschreckenden Wahrheit, dass sie sich in einer gef\u00e4hrlichen Lage befinden. Seine Fragen \u00fcber den Zustand der Dinge spiegeln seine innere Unruhe wider.\n- Der Protagonist, Dusk, analysiert die Situation und versucht, Caleor zu beruhigen, w\u00e4hrend er die Dringlichkeit der Flucht vor den Werw\u00f6lfen betont.\n- Caleor wird von der schockierenden Wahrheit \u00fcber das Werwolf-Gift \u00fcberw\u00e4ltigt, was zu einem emotionalen Konflikt zwischen Wut und Angst f\u00fchrt. Dies dr\u00e4ngt ihn, die Verantwortung f\u00fcr die Situation zu \u00fcbernehmen.\n- Die Gruppe erkennt die Notwendigkeit von Caleors Unterst\u00fctzung, um die Mission zu erf\u00fcllen. Dusk f\u00fchlt sich erleichtert, dass Caleor bereit ist, ihnen zu helfen, und st\u00e4rkt den Zusammenhalt unter den Freunden.\n- Sie positionieren sich strategisch am Waldfriedhof und bereiten sich auf die bevorstehende Konfrontation mit den Werw\u00f6lfen vor, w\u00e4hrend der Regen eine sch\u00fctzende Ablenkung bietet.\n- Pl\u00f6tzlich \u00f6ffnet sich das Portal, und die erste Welle der Werw\u00f6lfe tritt hervor, was die unmittelbare Bedrohung sp\u00fcrbar macht. Die Situation wird angespannt, als sie erkennen, dass sie schnell handeln m\u00fcssen.\n- Dusk beobachtet die Ankunft der Werw\u00f6lfe und stellt fest, dass sie Schwierigkeiten haben werden, ihren Plan ohne mehr Informationen \u00fcber Versiporcus umzusetzen.\n- Trotz der gef\u00fchlten Ausweglosigkeit motiviert Caleor die Gruppe, durchzuhalten, und sie setzen den Plan in Bewegung, um die Werw\u00f6lfe heimlich zu verfolgen.\n- Raven \u00fcbernimmt die Ablenkung, w\u00e4hrend Dusk und Caleor das Gewehr laden, was einen letzten verzweifelten Versuch darstellt, die Situation zu kontrollieren.\n- Die Spannung steigt, als sie die Werw\u00f6lfe verfolgen, und Dusk ist bereit, seine Sch\u00fcsse abzugeben, erkennt jedoch, dass die Gelegenheit zu schl\u00fcpfrig ist, um richtig zu handeln.\n- Die Konfrontation erreicht ihren H\u00f6hepunkt, als Versiporcus und Custos pl\u00f6tzlich auftauchen, was Caleors Panikattacke ausl\u00f6st und das \u00dcberleben der Gruppe auf die Probe stellt.
"""

raw_scene = generate_scene(raw_model, raw_tokenizer, scene_beats)
fine_tuned_scene = generate_scene(fine_tuned_model, fine_tuned_tokenizer, scene_beats, temp=0.5)

# Generate scenes
print("Raw model:")
print(raw_scene)
print("\n\n")
print("Fine-tuned model:")
print(fine_tuned_scene)

Prompt: "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nDu bist der Fantasy-Autor Yvo K. Jedes Mal, wenn ich dir einen Scene Beat vorschreibe, schreibst du die komplette Szene auf der Grundlage der Idee. Schliesse die Szene nicht selbst ab, sondern halte dich genau an die Scene Beats. Schliesse nicht mit einer Vorahnung ab.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n- Raven kniet verzweifelt im Schlamm und versucht, Caleor aus seiner Ohnmacht zu holen. Die Angst um Cale und die bevorstehende Gefahr treibt ihn an, w\u00e4hrend er gleichzeitig von der dr\u00e4ngenden Realit\u00e4t \u00fcberw\u00e4ltigt ist.\n- Caleor erwacht, panisch und desorientiert, konfrontiert mit der erschreckenden Wahrheit, dass sie sich in einer gef\u00e4hrlichen Lage befinden. Seine Fragen \u00fcber den Zustand der Dinge spiegeln seine innere Unruhe wider.\n- Der Protagonist, Dusk, analysiert die Situation und versucht, C

In [12]:
fine_tuned_scene = generate_scene(fine_tuned_model, fine_tuned_tokenizer, scene_beats, temp=0.4)

Prompt: "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou are the fantasy author Yvo K. Each time I prompt you with a scene beat, write the full scene based on the idea. Do not conclude the scene on your own, follow the beat instructions closely. Do not end with foreshadowing.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n- Kontext: Fantasy Roman ANSTURM\n- Charaktere: Raven, Dusk, Caleor, alles Jungs in der 8. Klasse.\n- Perspektive: 1. Person Sicht Caleor, m\u00e4nnlich, Ich-Erz\u00e4hler\n- Zeitform: Gegenwart\n- Raven \u00fcbernimmt die Kontrolle und stellt eine zynische Frage, die die Unlogik der Situation hervorhebt. Er ist frustriert und handelt impulsiv, indem er die Schl\u00fcssel an sich rei\u00dft und die T\u00fcr zuschl\u00e4gt, bevor er entschlossen Richtung B\u00fcro des Schulleiters geht.\n- Die restliche Gruppe bleibt zur\u00fcck. Der Protagonist Caleor lenkt die Aufmerksamkeit au

In [8]:
# print scene beats
print(scene_beats)


- Raven kniet verzweifelt im Schlamm und versucht, Caleor aus seiner Ohnmacht zu holen. Die Angst um Cale und die bevorstehende Gefahr treibt ihn an, während er gleichzeitig von der drängenden Realität überwältigt ist.
- Caleor erwacht, panisch und desorientiert, konfrontiert mit der erschreckenden Wahrheit, dass sie sich in einer gefährlichen Lage befinden. Seine Fragen über den Zustand der Dinge spiegeln seine innere Unruhe wider.
- Der Protagonist, Dusk, analysiert die Situation und versucht, Caleor zu beruhigen, während er die Dringlichkeit der Flucht vor den Werwölfen betont.
- Caleor wird von der schockierenden Wahrheit über das Werwolf-Gift überwältigt, was zu einem emotionalen Konflikt zwischen Wut und Angst führt. Dies drängt ihn, die Verantwortung für die Situation zu übernehmen.
- Die Gruppe erkennt die Notwendigkeit von Caleors Unterstützung, um die Mission zu erfüllen. Dusk fühlt sich erleichtert, dass Caleor bereit ist, ihnen zu helfen, und stärkt den Zusammenhalt unter 

In [9]:
text = "Dusk  Offenbarung\nSonntag, 5. Oktober 2014\n\n\u00bbCale! Caleor Rekon! Kannst du mich h\u00f6ren?!\u00ab Raven ist mit der Situation v\u00f6llig \u00fcberfordert, kniet im Schlamm und versucht ihn verzweifelt wachzur\u00fctteln. \u00bbNeeiin, Cale!\u00ab\n\u00bbVerflucht, Raven. Trest muss sich vertan haben. Es sollte doch fr\u00fchestens dienstags passieren!\u00ab\n\u00bbWir h\u00e4tten es ihm sagen m\u00fcssen!\u00ab Wasser tropft von Ravens durchn\u00e4ssten Haaren und l\u00e4uft ihm \u00fcber sein bleich gewordenes Gesicht. Er schaut best\u00fcrzt zu mir hoch.\n\u00bbKonnten wir nicht, sonst w\u00e4re er niemals mitgekommen. Wir brauchen ihn... Aber was tun wir jetzt?\u00ab Ich werfe einen kurzen Blick auf mein Handy. Die N\u00e4sse ist inzwischen bis in meine Hosentaschen vorgedrungen. Ich kann nur hoffen, dass es seinen Dienst nicht verweigern wird. \u00bbSie kommen in wenigen Minuten aus dem Portal. Bis dahin m\u00fcssen wir uns versteckt haben.\u00ab\n\nCaleor st\u00f6hnt unerwartet auf. Ich beuge mich neben Raven zu ihm herunter. Er scheint eine Weile zu brauchen, um zu realisieren, wo er sich befindet. Seine Augen wirken leer. Dann zuckt er zusammen und schaut mich fassungslos an.\n\u00bbWo ist er!? Wo sind sie?\u00ab Er atmet flach und schnell. \u00abSind wir tot?\u00ab Caleor setzt sich auf und zieht sich darauf hastig am Stamm des Baumes neben sich hoch. \u00bbAaaah\u00ab. Er verzerrt das Gesicht und fasst mit seiner rechten Hand an die schmerzende Stelle an seinem R\u00fccken, wo er zuvor auf die Wurzel gefallen ist. Unruhig blickt er um sich, versucht etwas zwischen den dunkeln B\u00e4umen zu erkennen.\n\u00bbVersuch dich erst mal zu beruhigen. Wir leben, aber jetzt hei\u00dft es vorsichtig sein.\u00ab Raven reicht Cale seine Wasserflasche, er trinkt einen gro\u00dfen Schluck. Langsam scheint er sich wieder unter Kontrolle zu haben. Mit dem Handr\u00fccken wischt er sich den gr\u00f6bsten Dreck vom Sturz aus seinem Gesicht. \n\u00bbHey, jetzt aber im Ernst: Was war das!? Ein Albtraum? Bin ich eingeschlafen?\u00ab\nIch presse meine Lippen zusammen und antworte ihm nach kurzem \u00dcberlegen. \u00bbDarauf kommen wir nachher zu sprechen. Es ist jedenfalls nicht auszuschlie\u00dfen, dass es erneut passiert. Versuch daran zu denken, dass nichts davon echt ist. Halte dich an den Dingen fest, wovon du dir sicher bist, dass sie real sind. Und jetzt folgt mir, wir m\u00fcssen weg von hier.\u00ab Ich f\u00fchre sie auf eine leicht erh\u00f6hte, um die hundert Meter vom Portal entfernte Stelle. \nCaleor scheint angestrengt nachzudenken. Nachdem er f\u00fcr l\u00e4ngere Zeit nichts mehr gesagt hat, richtet er sich energisch an uns. \u00bbIhr wisst mehr. Wieso habt ihr nicht erz\u00e4hlt, dass mir das passieren wird? Werde ich mich verwandeln!?\u00ab Die Wut steht ihm ins Gesicht geschrieben, gleichzeitig wird er von der Angst beherrscht, sich bald in einen Werwolf zu verwandeln. Absolut nachvollziehbar.\n\nEs f\u00fchrt kein Weg an der Wahrheit vorbei. Ich drehe mich langsam zu ihm um und antworte mit ernster Stimme. \u00bbCale. Damals, nachts am See. Ein Werwolf h\u00e4tte dich beinahe am Fu\u00df erwischt, als du vom gefallenen Baum in den See springen wolltest. Er hat dir einen Schuh weggerissen, aber du hattest Gl\u00fcck im Ungl\u00fcck. Es waren einzig feine Druckstellen seiner Rei\u00dfz\u00e4hne an deiner Ferse zur\u00fcckgeblieben.\u00ab\nRaven, der sich mittlerweile auch wieder beruhigt hat, f\u00e4hrt mit leiser Stimme fort. \u00bbDu wurdest zwar nicht gebissen, aber schon diese kleine Menge des \u00fcbertragenen Giftes hat eine immens starke Wirkung. Trest hat uns noch in derselben Nacht dar\u00fcber in Kenntnis gesetzt, als wir den Druckpunkt entdeckten. Er war es, der dich aus dem Wasser gerettet hat. Ohne ihn w\u00e4rst du ertrunken. Diese \u00c4ngste werden dich von nun an immer wieder heimsuchen. Von Zeit zu Zeit steigerst du dich in eine Paranoia hinein, und bildest dir ein, dass sie dich verfolgen. In Wirklichkeit spielt dir dein Verstand nur etwas vor. Es wird dich f\u00fcr den Rest deines Lebens begleiten, Cale. Tut mir leid.\u00ab\nCaleor sch\u00fcttelt fassungslos und ungl\u00e4ubig seinen Kopf. Er kann es nicht glauben. \u00bbDas war es also, was ihr mir verschwiegen habt. Ich dachte, wir k\u00f6nnten einander vertrauen. Wie konntet ihr mir das antun!? Was hattet ihr \u00fcberhaupt davon, mir diese Informationen vorzuenthalten?\u00ab\n\u00bbWir wollten nur das Beste f\u00fcr dich, das musst du uns glauben. H\u00e4tten wir dir davon erz\u00e4hlt, h\u00e4ttest du die letzten zwei Wochen nur noch darauf gewartet, dass es passiert. Es w\u00e4ren dir zwei Wochen weniger geblieben, in denen du ein normales Leben f\u00fchren konntest.\u00ab\nRaven nickt und erg\u00e4nzt meine Erkl\u00e4rung. \u00bbDu musst uns helfen. Wir konnten es uns nicht leisten, dich zu verlieren. Der Mutige, der Waghalsige, der Starke. Von uns dreien warst das alles immer du. Dumm ausgedr\u00fcckt sind wir nur Beilage, die ihr Bestes dazu beitr\u00e4gt, Sokrates Vorhaben zum Erfolg zu f\u00fchren. Um Klartext zu reden: Ohne dich schaffen wir es schlicht und einfach nicht, Caleor.\u00ab\nCale starrt niedergeschlagen auf den Boden und bleibt f\u00fcr eine halbe Ewigkeit in seine Gedanken vertieft. Raven und mir bleibt derweil nichts anderes \u00fcbrig, als seine Antwort abzuwarten. Ohne eine Idee davon, wie er darauf reagieren wird. Sollte er gehen wollen, k\u00f6nnen wir es ihm nicht \u00fcbel nehmen. So schwer es f\u00fcr uns dann auch werden w\u00fcrde, wir m\u00fcssten es alleine versuchen.\nSchlie\u00dflich schluckt Cale leer und schaut uns entschlossen an. \u00bbIch habe vermutet, dass diese urpl\u00f6tzlich aufgetauchten \u00c4ngste nicht nat\u00fcrlichen Ursprungs sein k\u00f6nnen. So schwer das f\u00fcr mich auch zu begreifen ist, passiert ist es bereits. Ihr habt nichts daf\u00fcr, dass es so gekommen ist.\u00ab Er legt eine kurze Pause ein und \u00fcberfliegt mit einem fl\u00fcchtigen Blick die B\u00e4ume um uns herum. \u00bbWelchen Grund h\u00e4tte ich also, mich zu weigern... Es soll nicht an mir scheitern. Ich will euch helfen.\u00ab\nMir f\u00e4llt ein Stein vom Herzen. Er hat es besser aufgenommen, als ich bef\u00fcrchtet hatte. Ein breites Grinsen \u00fcberzieht mein Gesicht. \u00bbIch bin froh, dass wir dich dabeihaben, Kumpel. Lass es uns durchziehen, was auch immer uns bevorstehen mag.\u00ab \nAuch Raven ist erleichtert. Er gibt Caleor einen Klaps auf die Schulter und dr\u00fcckt ihm anschlie\u00dfend seinen Rucksack in die H\u00e4nde. Das Wetter meint es gar nicht gut mit uns. Wenn die Temperaturen weiter so fallen, k\u00f6nnte es diese Nacht den ersten Schnee dieser Saison geben.\n\nWir  haben uns am Rande des Waldfriedhofes eine Stelle ausgesucht, die durch dichtes Gestr\u00fcpp getarnt ist. Hier warten wir auf das Eintreffen der Werw\u00f6lfe. Es d\u00fcrfte nicht mehr lange dauern, bis sich das Portal \u00f6ffnet. Nach Sokrates Aussage werden die Trupps einzeln durch das Dimensionstor kommen und umgehend die Richtung ihres Zieles anvisieren. Bei all seinen Nachteilen hat der Regen auch Gutes an sich. Durch das Prasseln, das die schweren Tropfen bei ihrem Aufprall verursachen, werden uns die Werw\u00f6lfe nicht so schnell h\u00f6ren. Ebenso werden jegliche Ger\u00fcche gleich wieder aus der Luft gewaschen, was uns ihre Verfolgung bedeutend vereinfachen sollte. Sollte es uns gelingen, heute Nacht in den Besitz des Amuletts von Versiporcus zu kommen, h\u00e4tten wir einen wesentlichen Teil zur Rettung tausender Menschen beigetragen.\n\nEin \u00fcberirdischer Laut geht vom alten Grabstein aus. Der Klang l\u00e4sst mir das Blut in den Adern gefrieren. Alle B\u00fcsche, die um den Stein herum wachsen, weichen wie von Geisterhand zur Seite. Aus meinem Rucksack dringt ein immer st\u00e4rker werdender, heller Schimmer unserer Amulette. Die dunkeln Schwaden breiten sich aus, das Portal w\u00e4chst nach und nach zu seiner vollen Gr\u00f6\u00dfe an. Sie kommen. Die Schwaden vermischen sich mit silbern schimmernden Funken, welche die Umgebung des Friedhofes in fahles Licht tauchen. Gerade genug hell, um die gr\u00f6bsten Konturen erkennen zu k\u00f6nnen. Der erste Werwolf tritt aus dem Portal, ein Amulett baumelt um seinen Hals. Gewissenhaft schaut sich der kr\u00e4ftige Wolf um. \n\u00bbDas muss Dextra sein.\u00ab, fl\u00fcstert Raven. \nEr wird sich versichern wollen, dass der Friedhof inzwischen nicht entdeckt wurde und belagert wird. F\u00fcr einen Moment taucht er wieder in die dichten Schwaden ein, kehrt dann zur\u00fcck und stellt sich parallel zum Portal hin. Wenig sp\u00e4ter kommen im Sekundentakt Werw\u00f6lfe dazu. Sie sammeln sich zu Truppen. Sobald eine komplett ist, verschwinden sie angef\u00fchrt von ihrem Leiter im Wald. Jede Gruppe wird bei ihrer R\u00fcckkehr nach dieser Nacht mindestens doppelt so viele Mitglieder umfassen. Allein der Gedanke daran l\u00e4sst mich erschaudern. Wir werden nicht zu den Opfern geh\u00f6ren.\nSeit mindestens zehn Minuten schauen wir nun bereits zu, wie unabl\u00e4ssig Werw\u00f6lfe das Portal verlassen und zwischen den B\u00e4umen verschwinden. Es m\u00fcssen hunderte sein. Wie sollen wir hier zu dritt etwas ausrichten? Die Lage scheint aussichtslos. Wir wissen nicht mal, ob Versiporcus bereits durch das Portal gekommen ist. Er k\u00f6nnte sich problemlos unter die anderen W\u00f6lfe gemischt haben. Wir h\u00e4tten ihn nicht erkannt. Keiner von uns hat ihn je gesehen.\nCaleor wirft eine Bemerkung dazwischen. \u00bbWir kennen Versiporcus zwar nur von Sokrates Beschreibung, aber sein Auftreten wird uns zeigen, ob wir es mit dem richtigen Werwolf zu tun haben\u00ab.\nIn diesem Moment tritt der letzte Werwolf aus den Portalschwaden und verschwindet, danach passiert nichts mehr. Ich beginne mich zu Fragen, ob wir Versiporcus vielleicht verpasst haben. Was uns jedoch weiter hoffen l\u00e4sst, ist Dextra. Er steht noch immer geduldig auf derselben Stelle und scheint etwas abzuwarten. Die K\u00e4lte dringt immer tiefer unter meine Jacke. Das wir hier stillsitzen m\u00fcssen, verbessert die harten Bedingungen \u00fcberhaupt nicht. Die Zeit vergeht. Gerade als ich kurz davor bin, die Geduld zu verlieren, tritt der Erwartete aus dem Portal. Im Vergleich zu anderen Werw\u00f6lfen ist er eher klein, und trotzdem m\u00e4chtiger als jeder andere. Begleitet wird er von einem seiner Leibw\u00e4chter. Jetzt gilt es ernst. Ich \u00f6ffne vorsichtig meinen Rucksack und hole die zwei Amulette heraus, die uns Sokrate gebracht hat. Einer von uns dreien wird ohne auskommen m\u00fcssen. Dieser eine werde ich sein. Ravens Einf\u00e4lle k\u00f6nnten uns noch n\u00fctzlich werden, und Caleor ist durch seine unkontrollierbaren Panikattacken schon genug im Nachteil.\n\nVersiporcus wendet sich ungeduldig an Dextra. \u00bbHast du ihn gesehen!? Hat er das Portal verlassen?\u00ab\n\u00bbNein. Wie ich vermutet habe, versteckt er sich nach wie vor. Es w\u00e4re auch dumm von ihm, sich zu zeigen. Er wei\u00df, was ihn im Falle seines Erscheinens erwartet. Aber diese Unannehmlichkeit soll nicht den Ansturm der heutigen Nacht beeintr\u00e4chtigen. Du sollst mit eigenen Augen sehen, welch gute Arbeit all deine Untergebenen leisten. Ihr begleitet den Trupp, der auf dem Weg zum nahe gelegenen Dorf ist. Custos wird dich zu ihm f\u00fchren. Ich halte hier die Stellung.\u00ab \nVersiporcus nickt und gibt seinem Leibw\u00e4chter Custos barsch zu verstehen, dass er ihn nun hinbringen soll. \n\nDas eine Amulett gebe ich Raven, das andere dr\u00fccke ich Caleor in die H\u00e4nde. Wir m\u00fcssen uns an ihre Fersen heften, ohne bemerkt zu werden. Niemand von uns hat auch nur die geringste Ahnung, wie wir an sein Amulett kommen, ohne dabei zum Werwolf zu mutieren. Die einzige uns zur Verf\u00fcgung stehende Waffe ist das Gewehr. Es muss jedoch immer wieder nachgeladen werden. Wenn uns keine List einf\u00e4llt, bis der Moment da ist...\nVersiporcus und sein Leibw\u00e4chter verschwinden im Wald Richtung Dorf. Ich werfe mir meinen Rucksack \u00fcber die Schultern und stehe auf. Raven und Caleor tun es mir gleich. Wir gehen in einem Bogen um das Portal herum, Dextra darf uns auf keinen Fall sehen. Der Weg f\u00fchrt uns hinter der kleinen Erh\u00f6hung nach unten und dann mitten durch den dichten Wald und das Gestr\u00fcpp Richtung Dorf. \nImmer wieder peitschen mir tiefh\u00e4ngende \u00c4stchen ins Gesicht, von der Dunkelheit verborgen sind sie kaum zu sehen. Es stellt sich als schwer heraus, mit den beiden Werw\u00f6lfen Schritt zu halten. Sie durchqueren den Wald in z\u00fcgigem Tempo, wir versuchen einen konstanten Abstand von ungef\u00e4hr hundert Metern einzuhalten. Dennoch fallen wir immer weiter zur\u00fcck. \nRaven tritt unerwartet auf eine nasse Wurzel, rutscht auf ihrer glitschigen Oberfl\u00e4che aus, f\u00e4llt in den nebenstehenden Busch und bricht dabei lautstark einige dickere \u00c4ste ab. Er landet mit den H\u00e4nden voraus im Schlamm und flucht leise vor sich hin. Mit ihren gro\u00dfen Pranken haben die Werw\u00f6lfe einen viel besseren Halt auf dem schlammigen Untergrund. Wir befinden uns an einer Stelle, wo das Gel\u00e4nde eben ist. Instinktiv springe ich hinter einem dicken Baum in Deckung. H\u00e4tten Versiporcus und sein Leibw\u00e4chter etwas von seinem Sturz mitbekommen und w\u00fcrden sich umdrehen, s\u00e4hen sie uns bestimmt. \nWir warten einige Sekunden ab, scheinen aber Gl\u00fcck gehabt zu haben. Weder Custos noch Versiporcus rechnet mit Verfolgern, sie setzen ihren Weg unbeirrt fort. Kurz darauf wage ich mich erleichtert aus der Deckung, der Regen scheint gen\u00fcgend laut gewesen zu sein, um die Ger\u00e4usche auf Distanz zu schlucken. Mittlerweile hat Raven sich wieder aufgerappelt. Ich achte darauf, mich m\u00f6glichst ruhig zu bewegen und erkundige mich bei Cale, der mit schmerzverzerrtem Gesicht an einem Baum lehnt. \u00bbGeht es?\u00ab\n\u00bbJa, im Moment jedenfalls. Ich gebe euch Bescheid, sollte ich wieder eine Attacke kommen sp\u00fcren. Was mir mehr zu schaffen macht, sind die Schmerzen im R\u00fccken. Es f\u00fchlt sich so an, als w\u00fcrde mein Rucksack von Schritt zu Schritt schwerer werden...\u00ab\n\u00bbWenn ich ihn dir abnehmen soll, sag bitte Bescheid. Wir sind aufeinander angewiesen. Wir k\u00f6nnten es uns nicht leisten, dich zur\u00fccklassen zu m\u00fcssen.\u00ab\n\nWir nehmen die Verfolgung wieder auf und bewegen uns dabei vorsichtiger denn je fort. Nach und nach n\u00e4hern wir uns dem Waldrand. Je nachdem, wie nah sich Versiporcus an das Geschehen heranbegeben wird, werden wir bald keine M\u00f6glichkeit mehr haben, ihn zu \u00fcberw\u00e4ltigen, ohne dass weitere Werw\u00f6lfe etwas davon mitbekommen. Zwei Werw\u00f6lfe sind schon eine riesige Herausforderung, mit mehr w\u00fcrden wir wohl kaum klarkommen. Wenn wir als Sieger nach Hause gehen wollen, m\u00fcssen wir jetzt etwas unternehmen. \nRaven scheint einen \u00e4hnlichen Gedanken gehabt zu haben und bringt einen Vorschlag. \u00bbIch lenke sie ab, Dusk. K\u00fcmmere du dich w\u00e4hrenddessen um das Gewehr.\u00ab\n\u00bbEinverstanden. Cale, behalte du die Lage im \u00dcberblick.\u00ab\n\nCale und ich bleiben zur\u00fcck um das Gewehr zu laden, w\u00e4hrend Raven sich an die Fersen der beiden Werw\u00f6lfe heftet. Munition haben wir bereits eingef\u00fcllt, nachdem wir die Funktionsweise des Gewehres inspiziert hatten. Ich nehme es aus dem Rucksack und ziehe den Entsicherungsriegel zur\u00fcck. Ein signalisierendes Klicken ert\u00f6nt. Das Gewehr ist, im Gegensatz zu mir, schussbereit. In den n\u00e4chsten Minuten kann so viel schief gehen... Eine M\u00f6glichkeit zum \u00dcben hatten wir nicht, der dadurch entstandene L\u00e4rm w\u00e4re wohl kaum unbemerkt geblieben. Ich habe nur sechs Sch\u00fcsse zur Verf\u00fcgung, um Patronen nachzuladen wird uns keine Zeit bleiben.\nMit dem Gewehr unter dem Arm renne ich in die Richtung, in die Raven verschwunden ist. Cale folgt dicht hinter mir. Diesmal geben wir uns keine M\u00fche mehr, leise zu sein. Der Regen hat sich inzwischen weiter verst\u00e4rkt, zunehmend bieten auch die B\u00e4ume keinen wirklichen Schutz vor dem fallenden Wasser mehr. W\u00e4hrend ich renne, sinken meine Schuhe immer wieder tief in den Schlamm ein. Ich entdecke Raven ein paar B\u00e4ume weiter links von uns, sein Blick ruht unsicher auf den beiden Werw\u00f6lfen, die noch maximal sechzig Meter von uns entfernt sind. Sie haben angehalten und scheinen angeregt miteinander zu diskutieren. Gerade so laut, dass es Cale und Dusk noch h\u00f6ren k\u00f6nnen, melde ich mich zu Wort. \u00bbHalt. Raven, warte. Wir sollten nicht n\u00e4her herangehen. Ich werde von hier aus versuchen, einen von ihnen zu treffen.\u00ab\n\u00bbUnd dann?\u00ab Raven hat den Kopf schiefgelegt und schaut mich fragend an, Wasser l\u00e4uft in B\u00e4chen von seinen Haaren.\n\u00bbDann improvisieren wir. Eine andere M\u00f6glichkeit sehe ich nicht.\u00ab\nEr nickt zustimmend und wendet seinen Blick anschlie\u00dfend wieder Versiporcus und seinem Leibw\u00e4chter zu. Ich gehe ein paar Meter weiter nach vorne und knie mich hinter einem alten, mit Moos bewachsenen Baumstrunk auf den Boden. Den Lauf des Gewehres lege ich auf das morsche Holz, mit der linken Hand halte ich ihn in Position. Meine rechte Hand richtet das Gewehr so aus, dass es auf Versiporcus zielt, den kleineren der beiden Werw\u00f6lfe. Meine zittrigen Finger n\u00e4hern sich dem Ausl\u00f6ser. Gerade als ich abdr\u00fccken will, drehen die Werw\u00f6lfe ab und setzen ihren Weg ins Dorf fort. Ich habe zu lange gez\u00f6gert. Wir haben den richtigen Moment verpasst und unsere wahrscheinlich einzige Chance vergeben. Verzweifelt l\u00f6se ich die Anspannung in meinen Armen, lasse das Gewehr wieder sinken und schaue zu Cale und Raven zur\u00fcck. Mindestens genauso niedergeschlagen wie ich es bin, l\u00e4sst Cale seinen Kopf sinken.\n\nRaven bei\u00dft sich auf die Unterlippe und denkt eine Weile nach. \u00bbWir haben Sokrate versprochen, zu helfen. Vielleicht bekommen wir eine zweite Chance. Wir sollten abwarten.\u00ab\nFrierend und mit einem unbehaglichen Gef\u00fchl im Magen setzen wir uns auf den Stamm einer umgefallenen Tanne und warten ab, w\u00e4hrend der Regen den ganzen Waldboden rund um uns herum nach und nach in eine einzige Schlammpf\u00fctze verwandelt. Altes Laub wird vom Wasser durch viele in den Untergrund gesp\u00fclte Furchen an tiefere Stellen des Waldes geschwemmt. Die K\u00e4lte dringt immer tiefer unter die Kleidung vor, ich zittere am ganzen K\u00f6rper und versuche durch stetiges Reiben wenigstens meine H\u00e4nde warm zu halten. Nach und nach wandeln sich die schweren Regentropfen zu Eiskristallen, welche das Licht meiner Stirnlampe teilen und Tausende kleine, tanzende Lichtpunkte auf die umstehenden B\u00e4ume werfen.\n\nRaven springt abrupt auf und leuchtet mit seiner Taschenlampe fl\u00fcchtig in das Waldst\u00fcck hinter uns. Entgeistert starrt er auf eine Stelle direkt hinter mir und Cale. Ein Ast knackt, und gleich darauf nehme ich Schritte wahr. Pranken, die sich \u00fcber den matschigen Untergrund direkt auf uns zu bewegen. Ich packe das geladene Gewehr, sto\u00dfe mich mit dem rechten Fu\u00df vom Baumstamm ab und drehe mich auf der Stelle. Versiporcus und Custos haben sich unbemerkt an uns herangeschlichen. Cales Gesicht zeigt einen Ausdruck von Angst, wie ich ihn noch nie zuvor auch nur Ansatzweise gesehen habe. In einer unverst\u00e4ndlichen Sprache h\u00f6re ich Versiporcus etwas sagen. Seine Augen leuchten h\u00e4misch. Ohne Amulett kann ich ihn nicht verstehen, doch nach Ravens Reaktion zu schlie\u00dfen, haben seine Worte nichts Gutes zu bedeuten. Die beiden Werw\u00f6lfe treten immer n\u00e4her zu uns heran, ich weiche stolpernd zur\u00fcck. Eine neue Panikattacke \u00fcbermannt Caleor, er kann sich nicht mehr l\u00e4nger auf seinen Beinen halten und f\u00e4llt seitlich zu Boden. Reglos bleibt er liegen, direkt vor den F\u00fcssen des skrupellosen Anf\u00fchrers der Werw\u00f6lfe. Versiporcus richtet seinen Blick auf Caleor und gibt einen h\u00f6hnischen Laut von sich.\u2003"
print(text)

Dusk  Offenbarung
Sonntag, 5. Oktober 2014

»Cale! Caleor Rekon! Kannst du mich hören?!« Raven ist mit der Situation völlig überfordert, kniet im Schlamm und versucht ihn verzweifelt wachzurütteln. »Neeiin, Cale!«
»Verflucht, Raven. Trest muss sich vertan haben. Es sollte doch frühestens dienstags passieren!«
»Wir hätten es ihm sagen müssen!« Wasser tropft von Ravens durchnässten Haaren und läuft ihm über sein bleich gewordenes Gesicht. Er schaut bestürzt zu mir hoch.
»Konnten wir nicht, sonst wäre er niemals mitgekommen. Wir brauchen ihn... Aber was tun wir jetzt?« Ich werfe einen kurzen Blick auf mein Handy. Die Nässe ist inzwischen bis in meine Hosentaschen vorgedrungen. Ich kann nur hoffen, dass es seinen Dienst nicht verweigern wird. »Sie kommen in wenigen Minuten aus dem Portal. Bis dahin müssen wir uns versteckt haben.«

Caleor stöhnt unerwartet auf. Ich beuge mich neben Raven zu ihm herunter. Er scheint eine Weile zu brauchen, um zu realisieren, wo er sich befindet. Seine Augen